# Agriculture allocation and optimisation
Built on: 
+ Kondo, Y.; Nakamura, S. Waste input–output linear programming model with its application to eco-efficiency analysis. Econ. Systems Res. 2005, 17 (4), 393-408; DOI 10.1080/09535310500283526.
+ Gaustad, Olivetti, and Kirchain (2011): Toward Sustainable Material Usage: Evaluating the Importance of Market Motivated Agency in Modeling Material Flows
+ Løvik, A.N., Modaresi, R., Müller, D.B., 2014. Long-term strategies for increased recycling of automotive aluminum and its alloying elements. Environ. Sci. Technol. 48, 4257–4265.
+ Ohno et al. (2017, under review)
+ previous script _Waste management optimisation.ipynb_

## Problem statement
The following problem arose as part of a master thesis project on identifying optimal production patterns to fill a given food basket for a given region (Baden-Württemberg, Germany).

We have a given food basket w (w is a vector with entries such as 'protein', 'cereals', etc.). We have a number of farm types that produce food: group 1, the meat and dairy producers, contains m different farm types. Group 2, the crop producers, contains n different farm types. The total farm output is denoted by x. Farms themselves need agricultural input, e.g., grain to feed pigs. This input is listed in a matrix Z. Farms need resources such as fertilizer (r), land (l), and have costs or environmental impacts c. Each farm has one main output only. (By-products can be negative entries in Z). The final demand for farm products is y, which is mapped to w.

According to the data that were available the land constraint is somewhat tricky:
The total arable land is divided into different groups/classes, depending on what crops can be grown there. 
Some land can grow potatoes, wheat, rye, and sugar beets, other only potatoes and rye, for example.
Moreover, crop rotation represents a constraint, as one cannot always grow wheat on a given field.
Below the system definition:

<img src="Agriculture.png" width="504" height="490" alt="System definition">

__Figure 1:__ System definition.

The challenge here is that we not only have a single land constraints but different groups/classes of producing land.
We thus need to split the total output vector into an output vector of each land group. That means that the columns of the Z matrix also needs to be split, and both the split x and the split Z must enter the objective function.

The script _IEooc_Waste management optimisation.ipynb_, which is also part of this Github repository, contains a more detailed explanation and derivation of the method applied here. A working solution is that both x and Z must enter the objective function and a number of constraints is defined and stacked into a large constraint matrix.

__Introducing the model parameters and examples__

1) The basked of goods $w$ bought and consumed by the people. Building scenarios for $w$ is part of the project but not described here. 


In [1]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

FoodCats = ['Proteins','Meat','Cereals'] #Unit: kg
No_FoodCats = len(FoodCats)
w = np.array(([10,20,15])) # Basket of final goods

2) The list of meat products, the list of crops, the final demand for farm products $y$ and the mapping matrices from meat and crops to the food basket, $W_m$ and $W_c$.


In [2]:
Meats = ['Beef','Poultry','Pig','Lamb'] # in kg
No_Meats = len(Meats)
Crops = ['Grass','Wheat','Potatoes','Oats'] # in kg
No_Crops = len(Crops)
Landgroups = ['Pasture/Grassland','Field_1','Field_2','Field_3','Field_4']
No_Landgroups = len(Landgroups)

Wm = np.array(([[0.26,0.26,0.26,0.26],[1,1,1,1],[0,0,0,0]])) # number of FoodCats x number of meat products
Wc = np.array(([[0,0.1,0.1,0.1],[0,0,0,0],[0,1,0,1]])) # number of FoodCats x number of crops
print(Wm)
print(Wc)
W = np.concatenate((Wm,Wc), axis =1) # stack Wm and Wc to map expanded output vector to food basket.
print(W.shape)
No_x = No_Meats + No_Crops # Number of markets
print(No_x)
No_X = No_Meats + No_Crops * No_Landgroups # Length of expanded X vector
print(No_X)

[[ 0.26  0.26  0.26  0.26]
 [ 1.    1.    1.    1.  ]
 [ 0.    0.    0.    0.  ]]
[[ 0.   0.1  0.1  0.1]
 [ 0.   0.   0.   0. ]
 [ 0.   1.   0.   1. ]]
(3, 8)
8
24


3) The specific land requirement of crop production $l$, the land groups, the total available land by group $L$, and the maximal production share table for each land type $LS$.

In [3]:
l  = np.array(([0.1,0.15,0.18,0.25])) # land for field crops in area per kg

L  = np.array(([100,20,200,80,120])) # Total land (area units) for each land group

LS = np.array(([1,0,0,0],[0,0.5,0.25,0.25],[0,0,0.75,0],[0,0.4,0,0.4],[0,0,0.4,0.4])) # Land group by crop: maximal production share according to crop rotation
print(LS)
print((np.diag(L).dot(LS)))

[[ 1.    0.    0.    0.  ]
 [ 0.    0.5   0.25  0.25]
 [ 0.    0.    0.75  0.  ]
 [ 0.    0.4   0.    0.4 ]
 [ 0.    0.    0.4   0.4 ]]
[[ 100.    0.    0.    0.]
 [   0.   10.    5.    5.]
 [   0.    0.  150.    0.]
 [   0.   32.    0.   32.]
 [   0.    0.   48.   48.]]


4) The cost or GHG intensity vector c:

In [4]:
cm = np.array(([5,2,3,1.5]))       # For meat products
cc = np.array(([0.5,0.6,0.3,0.4])) # For crops

5) The intermediate demand $A$ of crops for meat production.

In [5]:
A = np.array(([12,5,1,4],[1.5,2,2,1.5],[2,3,2,3],[15,1,0,4])).transpose() # in kg per kg
print(A)
# dimensions of A: crop types (rows) by meat types (columns)

[[ 12.    1.5   2.   15. ]
 [  5.    2.    3.    1. ]
 [  1.    2.    2.    0. ]
 [  4.    1.5   3.    4. ]]


## Approaching a formulation as linear problem
The Python routine scipy.optimize.linprog offers a solver for a linear programme in the standard form

$$
\begin{eqnarray}
min \> C = c^t\cdot x  \\
\mbox{s.t.} \\
B\cdot x \leq b \\
B_{eq}\cdot x = b_{eq}  \\
lb \leq x \leq ub
\end{eqnarray}
$$

All subsequent linear programmes will be converted to that standard form, using suitable linear operators to build the constraint matrices.

The following objective function and constraints are used:

__1) Objective function and costs__

The total output vector $x$ is a stack of the meat output plus No_Landgroups * No_Crops different crop outputs.

len(x) = No_Meats + No_Landgroups * No_Crops

We have len(x) farm types and len(x) markets. The $Z$ matrix contains flows from meat and crop markets to farms in the different land groups, has the dimension (No_Meats + No_Crops) x (No_Meats + No_Landgroups * No_Crops).

The linear program will find both x and Z, and to do that we need to stack x and the columns of Z into a big objective vector X. 

We can then define a cost vector C of the same length as X.

__2) Final demand constraint__

The market balance reads
$$ y = x - Z\cdot e$$
where $e$ is a summation vector of only ones. x ist determined from X by summing up over the different land groups.

The food basket is determined from y as:
$$ w = W \cdot y$$
From which follows that
$$ w = W \cdot y = W \cdot x - W \cdot Z\cdot e$$
From which the first inequality contraint follows:
$$ W \cdot x - W \cdot Z\cdot e \geq w$$
or
$$ W \cdot Z\cdot e - W \cdot x   \leq -w$$
Now a constraint matrix has build for this equation that fits the format required by Python ($B\cdot X \leq b$), where X and Z are part of the same objective vector X. This matrix is built below.

__3) Intermediate demand constraint__

The intermediate demand $Z$ is proportional to the output vector $X$, and the proportionality factor is the so-called matrix of technical cofficients $A$, a term that is taken from Leontief-input-output analysis. The following relationship between $A$ and $Z$ holds:
$$ A \cdot \hat{X} = Z$$
or
$$ A \cdot \hat{X} \leq Z$$
$$ A \cdot \hat{X} - Z \leq 0$$
A matrix operator for this constraint is built below.

__4) Crop rotation constraint__

The matrix $\hat{L}\cdot LS$ shows the total land that is available for each crop in each land group. 
A matrix operator for this constraint is built below.

__5) Total land constraint__

The output vector X, multiplied with the specific land requirements l, must not be larger than the available land L.
A matrix operator for this constraint is built below.

__6) Sign constraints__

We need to stipulate that $y\geq 0$ and $x\geq 0$.
For y a matrix operator constraint will be built below, for x, the constraint can be specified when calling the solver.


## Helper functions for transforming matrix constraints into standard form


In [6]:
def MI_Tupel(value, Is): 
    """
    Define function for obtaining multiindex tuple from index value
    value: flattened index position, Is: Number of values for each index dimension
    Example: MI_Tupel(10, [3,4,2,6]) returns (0,0,1,4)
    """
    IsValuesRev = []
    CurrentValue = value
    for m in range(0,len(Is)):
        IsValuesRev.append(CurrentValue % Is[len(Is)-m-1])
        CurrentValue = CurrentValue // Is[len(Is)-m-1]
    return IsValuesRev[::-1]


def Summation_Matrix(To,Tn,Is):
    """
    Define function for summation matrix operator for multi-indizes
    This function returns a matrix, which, when applied to a flattened multidimensional array, gives the same result as the operation
    Y(dr,ds,dt, ...) = Sum(d1,d2,d3, ...) X(d1,d2,d3, ... , dr, ds, dt, ...)
    The order of the different dimensions can be arbitrary but needs to specified.
    Function takes three lists: index hierarchy old, index hierarchy new, index size
    example: To = ['w', 's', 'm', 'e'], Tn = ['w', 'm', 'e'], Is = [3,4,2,6]
    """    
    # Check input data consistency:
    if len(To) != len(Is):
        return 'Error: Length of old index hierarchy list must be the same as length of index size list.'
    
    # Determine hierarchy levels of indices in multiindex for new and old array
    OldPos = np.arange(0,len(To)).tolist()
    NewPos = []
    [NewPos.append(To.index(i)) for i in Tn]
    NewIs = [Is[To.index(i)] for i in Tn]
    # return NewPos
    Rows = int(np.prod([Is[i] for i in NewPos])) # number of rows of aggregation matrix
    Cols = int(np.prod([Is[i] for i in OldPos])) # number of columns of aggregation matrix
    # return (Rows, Cols)
    am = np.zeros((Rows,Cols)) # define empty aggregation matrix
    if Tn == []: # if all dimensions are summed up:
        for m in range(0,Rows):
            for n in range(0,Cols): # Fill aggregation matrix
                am[m,n] = 1
    else: # if summation matrix has more than one row
        for m in range(0,Rows):
            for n in range(0,Cols): # Fill aggregation matrix
                RowTupel = MI_Tupel(m,NewIs)
                ColTupel = MI_Tupel(n,Is)
                ColTupel_inNewPos = [ColTupel[i] for i in NewPos]
                if RowTupel == ColTupel_inNewPos:
                    am[m,n] = 1
    return am


def Weighted_Partial_Summation_Matrix(WeightArray,IndxNew,IndxArray,IndxOld,SizeArray,SizeOld):
    """
    Define function for weighted partial summation matrix operator for multi-indizes
    This function returns a matrix, which, when applied to a flattened multidimensional array, gives the same result as the operation
    Y(dr,ds,dt, ..., da, db, dc, ...) = Sum(d1,d2,d3, ...) W(d1,d2,d3, ... , da, db, dc) \cdot X(d1,d2,d3, ... , dr, ds, dt, ...)
    The order of the different dimensions can be arbitrary but needs to specified.
    Function takes the following arguments: multidim. weighting factor array with five lists: IndxOld: hierarchy old of x vector, IndxArray: index hierarchy of weighting array, IndxNew: index hierarchy new, SizeOld: index sizes for x, and SizeArray: index sizes for weighting array
    example 1: WeightArray = n-dim array with indices IndxArray = ['w', 's', 'm', 'e', 'a'] and SizeArray = [3,4,2,6,3], objective vector x with multiindex IndxOld = ['w', 's', 'm', 'e'], and SizeOld = [3,4,2,6], new vector with multiindex  IndxNew = ['s', 'a']  
    example 2: a matrix term W*X*e, where W contains weights, X is the to-be-flattened matrix, and e is the row sum operator (vector of ones)
    Result = Weighted_Partial_Summation_Matrix(W,['i'],['i','k'],['k','j'],[],[])
    """
    
    # Check input data consistency:
    if len(IndxArray) != len(SizeArray):
        return 'Error: Length of old index hierarchy list must be the same as length of index size list for x vector.'
    if len(IndxOld) != len(SizeOld):
        return 'Error: Length of old index hierarchy list must be the same as length of index size list for x weighting array.'
    CommonDims = [i for i, j in zip(IndxArray, IndxOld) if i == j]
    # return CommonDims
    for m in CommonDims:
        if SizeArray[IndxArray.index(m)] != SizeOld[IndxOld.index(m)]:
            return 'Error: Dimemsion mismatch. At least one index has different dimensions in array and vector.'
    if tuple(SizeArray) != WeightArray.shape:
        return 'Error: Dimensions of SizeArray do not match the dimensions of the index provided.'
    
    # Determine properties of array family
    SizeNew = [] # Determine list of dimension sizes of new array
    for m in range(0,len(IndxNew)):
        if IndxNew[m] in IndxArray: # Check if this particular index is present in the array, and, if so, take its size
            SizeNew.append(SizeArray[IndxArray.index(IndxNew[m])])
        else:
            SizeNew.append(SizeOld[IndxOld.index(IndxNew[m])]) # Take size from objective vector x
    # return SizeNew
    
    CommonIndizesResultVector   = list(set(IndxNew) & set(IndxOld)) # List of index letters that the result vector and the original vector have in common. These indized are not summed over.
    # return CommonIndizesResultVector
    CommonIndexPositionResult   = [IndxNew.index(i) for i in CommonIndizesResultVector] # Position of common indices in result vector
    # return CommonIndexPositionResult
    CommonIndexPositionOriginal = [IndxOld.index(i) for i in CommonIndizesResultVector] # Position of common indices in original vector
    # return CommonIndexPositionOriginal
    
    Rows = int(np.prod(SizeNew)) # number of rows of aggregation matrix
    Cols = int(np.prod(SizeOld)) # number of columns of aggregation matrix
    # return (Rows, Cols)
    am = np.zeros((Rows,Cols)) # define empty aggregation matrix am  
    for m in range(0,Rows):
        for n in range(0,Cols): # Fill aggregation matrix
            RowTupel = MI_Tupel(m,SizeNew)
            ColTupel = MI_Tupel(n,SizeOld)
            # First, we check whether this Row/Column intersection has an entry at all. It does if the indices that the result vector and the original vector have in common have the same values.
            RowCommonIndices = [RowTupel[i] for i in CommonIndexPositionResult]
            ColCommonIndices = [ColTupel[i] for i in CommonIndexPositionOriginal]
            # print(RowCommonIndices, ColCommonIndices)
            if RowCommonIndices == ColCommonIndices: # The non-summation indices match, that means we will insert the value of the weighting array here.
                # Now, we need to get the indices for the Weighting Array from those tuples.
                ThisIndex = [] # list of indices for the dimensions of the weighting array for this entry
                for o in range(0,len(IndxArray)):
                    if IndxArray[o] in IndxNew:
                        ThisIndex.append(RowTupel[IndxNew.index(IndxArray[o])])
                    else:
                        ThisIndex.append(ColTupel[IndxOld.index(IndxArray[o])])
                # print(ThisIndex)
                am[m,n] = WeightArray[tuple(ThisIndex)]
    return am



# End

In [7]:
#TESTS:

#A = Summation_Matrix(To = ['w', 's', 'm', 'e'], Tn = ['w', 'm', 'e'], Is = [3,4,3,2])
#A = Summation_Matrix(To = ['w', 'm', 'e'], Tn = ['w', 'e'], Is = [3,2,1])
#A = Summation_Matrix(To = ['w', 's', 'm', 'e', 'b'], Tn = ['w', 'm', 'e'], Is = [3,4,3,2,6])
#A = Summation_Matrix(To = ['w', 's', 'm', 'e', 'b'], Tn = [], Is = [3,4,3,2,6])
# A = Summation_Matrix(To = ['w', 's', 'm', 'e'], Tn = ['w', 's', 'm', 'e'], Is = [3,4,3,2]) # yields unit matrix
#print(A.shape)
#print(A)

#B = Weighted_Partial_Summation_Matrix(WeightArray = np.random.random([3,4,2,6,3]), IndxArray = ['w', 's', 'm', 'e', 'a'],SizeArray = [3,4,2,6,3], IndxOld = ['w', 's', 'm', 'e'], SizeOld = [3,4,2,6], IndxNew = ['s', 'a', 'm'])
#B = Weighted_Partial_Summation_Matrix(WeightArray = np.random.random([2]), IndxArray = ['e'], SizeArray = [2], IndxOld = [ 'm', 'e'], SizeOld = [3,2], IndxNew = ['e'])
#B = Weighted_Partial_Summation_Matrix(WeightArray = np.random.random([2,4]), IndxArray = ['e','a'], SizeArray = [2,4], IndxOld = [ 'm', 'e'], SizeOld = [3,2], IndxNew = ['a'])
# print(B)
#print(B.shape)


## A field to crop/meat allocation procedure
We have a number of field groups with certain crop rotation patterns. We need meat products and crops to fill a given food basket. Parametes and constraints as defined above.

In [8]:
# A field to crop/meat allocation procedure
# Define indices and item lists
# --> Above

# Define input parameters
# --> Above

# OPTIMISATION MODEL

# 1) Objective function and costs
# X is a stack of the output vector x (which again is a stack of output by different field groups) and Z
len_x =  No_Meats + No_Crops * No_Landgroups # output vector x
print('Length of output vector: ' + str(len_x))
len_X =  len_x + len_x * (No_Meats + No_Crops) # Big objective vector X (x stacked on top of columns of Z)
print('Length of objective vector: ' + str(len_X))

# We define the cost or GHG vector C:
C    = np.zeros((len_X))
C[0:No_Meats] = cm # add costs for meat production
C[No_Meats:No_Meats+No_Crops*No_Landgroups] = np.tile(cc.reshape(No_Crops,1),(No_Landgroups,1)).flatten()# add costs for crop production by field group
#print(C)
print('Shape of C: ' + str(C.shape))

# Build inequality constraints

# 2) Final demand constraint
IneqCV1  = -w
IneqCM1a = -np.concatenate((Wm,np.tile(Wc,(1,No_Landgroups))), axis =1) # stack Wm and Wc to map expanded output vector to food basket.
#print(IneqCM1a.shape)
IneqCM1b = Weighted_Partial_Summation_Matrix(W,['i'],['i','k'],['k','j'],[No_FoodCats,No_x],[No_x,No_X])
#print(IneqCM1b.shape)
IneqCM1  = np.concatenate((IneqCM1a,IneqCM1b), axis =1)
print('Shape of final demand constraint: ' + str(IneqCM1.shape))

# 3) Intermediate demand constraint (as equality constraint)
EqCV1  = np.zeros((No_x *No_X,1))
EqCM1a = np.zeros((No_x *No_X,No_X))
for m in range(0,No_Meats):
    for n in range(0,No_Crops):
        EqCM1a[No_Meats*No_X +n*No_X +m,m] = A[n,m] 
EqCM1b = -np.eye(No_x *No_X)
EqCM1 = np.concatenate((EqCM1a,EqCM1b), axis =1)

# 4) Crop rotation constraint
IneqCV2 = (np.diag(L).dot(LS)).flatten()
IneqCM2 = np.zeros((No_Crops * No_Landgroups,len_X))
for m in range(0,No_Landgroups):
    IneqCM2[m*No_Crops:(m+1)*No_Crops,No_Meats +m*No_Crops:No_Meats + (m+1)*No_Crops] = np.diag(l)

# 5) Total land constraint
IneqCV3 = L
IneqCM3 = np.zeros((No_Landgroups,len_X))
for m in range(0,No_Landgroups):
    IneqCM3[m,No_Meats + No_Crops*m : No_Meats + No_Crops*(m+1)] = l # enter specific land requirements into constraint matrix
    
# 6) Final demand sign
IneqCV4  = np.zeros((No_x))
IneqCM4a = np.zeros((No_x,No_X))
IneqCM4a[0:No_Meats,0:No_Meats] = -np.eye(No_Meats)
for m in range(0,No_Landgroups):
    IneqCM4a[No_Meats::,No_Meats+m*No_Crops:No_Meats+(m+1)*No_Crops] = -np.eye(No_Crops)
#print(IneqCM4a.shape)
IneqCM4b = Summation_Matrix(To = ['a', 'b'], Tn = ['a'], Is = [No_x,No_X])
#print(IneqCM4b.shape)
IneqCM4  = np.concatenate((IneqCM4a,IneqCM4b), axis =1)
print('Shape of final demand sign constraint: ' + str(IneqCM4.shape))    

# Build inequality constraint
IneqCM = np.concatenate((IneqCM1,IneqCM2,IneqCM3,IneqCM4), axis =0) #,IneqCM4
IneqCV = np.concatenate((IneqCV1,IneqCV2,IneqCV3,IneqCV4), axis =0) #,IneqCV4

# Build equality constraint
EqCM = EqCM1
EqCV = EqCV1

# Call the solver
res = linprog(C, A_ub=IneqCM, b_ub=IneqCV, A_eq=EqCM, b_eq=EqCV, bounds=(0, None), options={"disp": True})
# Coeff. in objective function
print(res)

# Evaluate and check model results
x_meat = res['x'][0:No_Meats]
x_crop = res['x'][No_Meats:No_Meats + No_Crops*No_Landgroups].reshape(No_Landgroups,No_Crops)
Z_res  = res['x'][No_Meats + No_Crops*No_Landgroups::].reshape(No_x,No_X) # Result in matrix form

#
#
#

Length of output vector: 24
Length of objective vector: 216
Shape of C: (216,)
Shape of final demand constraint: (3, 216)
Shape of final demand sign constraint: (8, 216)
Optimization terminated successfully.
         Current function value: 118.900000  
         Iterations: 212
     fun: 118.89999999999998
 message: 'Optimization terminated successfully.'
     nit: 212
   slack: array([  97.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
          0.  ,    0.  ,    0.  ,  141.86,    0.  ,    0.  ,   32.  ,
          0.  ,   25.75,    0.  ,    0.  ,   48.  ,   48.  ,   97.  ,
          4.  ,  191.86,   73.75,  120.  ,    0.  ,   20.  ,    0.  ,
          0.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,   33.  ,   15.  ])
  status: 0
 success: True
       x: array([  0.        ,  20.        ,   0.        ,   0.        ,
        30.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  40.        ,  27.77777778,  20.        ,
         0.        ,   0.       

In [9]:
pd.DataFrame(data = x_meat, index = Meats, columns = ['Meat output in kg'])



,Meat output in kg
Beef,0
Poultry,20
Pig,0
Lamb,0


In [10]:
pd.DataFrame(data = x_crop, index = Landgroups, columns = Crops)

,Grass,Wheat,Potatoes,Oats
Pasture/Grassland,30,0,0.000000,0
Field_1,0,40,27.777778,20
Field_2,0,0,45.222222,0
Field_3,0,0,0.000000,25
Field_4,0,0,0.000000,0


In [11]:
y = np.concatenate((x_meat,x_crop.sum(axis =0)),axis =0) - Z_res.sum(axis =1)
pd.DataFrame(data = y, index = Meats+Crops, columns = ['Final demand, kg'])

,"Final demand, kg"
Beef,0.000000e+00
Poultry,2.000000e+01
Pig,0.000000e+00
Lamb,0.000000e+00
Grass,0.000000e+00
Wheat,-7.105427e-15
Potatoes,3.300000e+01
Oats,1.500000e+01


In [12]:
w_result = W.dot(y)
print(W)
pd.DataFrame(data = w_result, index = FoodCats, columns = ['Final demand, kg'])

[[ 0.26  0.26  0.26  0.26  0.    0.1   0.1   0.1 ]
 [ 1.    1.    1.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    1.    0.    1.  ]]


,"Final demand, kg"
Proteins,10
Meat,20
Cereals,15


In [13]:
Z_res.sum(axis =1)
Z_res

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.],
       [  0.,  30.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.],
       [  0.,  40.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
       

In [14]:
#Used and unused land:
TotalLandUsed        = np.diag(l).dot(x_crop.sum(axis =0))
TotalLandUsedByGroup = np.tile(l,(No_Landgroups,1)) * x_crop
TotalLandUnused      = L - TotalLandUsedByGroup.sum(axis =1)
pd.DataFrame(data = TotalLandUsed, index = Crops, columns = ['ha'])

,ha
Grass,3.00
Wheat,6.00
Potatoes,13.14
Oats,11.25


In [15]:
pd.DataFrame(data = TotalLandUsedByGroup, index = Landgroups, columns = Crops)

,Grass,Wheat,Potatoes,Oats
Pasture/Grassland,3,0,0.00,0.00
Field_1,0,6,5.00,5.00
Field_2,0,0,8.14,0.00
Field_3,0,0,0.00,6.25
Field_4,0,0,0.00,0.00


In [16]:
pd.DataFrame(data = TotalLandUnused, index = Landgroups, columns = ['ha'])

,ha
Pasture/Grassland,97.00
Field_1,4.00
Field_2,191.86
Field_3,73.75
Field_4,120.00


The above example shows that large problems can be quickly formulated with the helper functions defined above and solved. 